In [16]:
import tensorflow as tf
import numpy as np
from keras import models, layers

In [17]:
# Placeholder for input image
X = tf.keras.layers.Input(shape=(150, 150, 1), name="X", dtype=tf.float32)

In [18]:
# Define the convolutional parameters
conv1_parameters = {
    "filters": 16,
    "kernel_size": (5, 5),
    "strides": (1, 1),
    "padding": "same",
    "activation": tf.nn.relu
}
conv2_parameters = {
    "filters": 32,
    "kernel_size": (5, 5),
    "strides": (2, 2),
    "padding": "same",
    "activation": tf.nn.relu
}

conv3_parameters = {
    "filters": 64,
    "kernel_size": (5, 5),
    "strides": (1, 1),
    "padding": "same",
    "activation": tf.nn.relu
}

conv4_parameters = {
    "filters": 128,
    "kernel_size": (9, 9),
    "strides": (1, 1),
    "padding": "same",
    "activation": tf.nn.relu
}


In [19]:

# Create the four convolutional layers

conv1 = layers.Conv2D(name="conv1", **conv1_parameters)(X)
conv2 = layers.Conv2D(name="conv2", **conv2_parameters)(conv1)
conv3 = layers.Conv2D(name="conv3", **conv3_parameters)(conv2)
conv4 = layers.Conv2D(name="conv4", **conv4_parameters)(conv3)


In [20]:
def squash(s, epsilon=1e-7, name=None): 
    with tf.name_scope(name):
        squared_norm = tf.reduce_sum(tf.square(s), axis=-1, keepdims=True)
        safe_norm = tf.sqrt(squared_norm + epsilon)
        squash_factor = squared_norm / (1. + squared_norm)
        unit_vector = s / safe_norm
        return squash_factor * unit_vector


In [21]:
# Primary capsule layer 

primary_capsule_input = tf.reshape(conv4, [-1, 1152 , 8], name="primary_capsule_input")
primary_capsule_output = squash(primary_capsule_input, name="primary_capsule_output")

In [28]:
# Secondary Capsule layer 

# Initializing weight matrix 

W_init = tf.random.normal(shape=(1, 1152, 3, 16, 8), stddev=0.1, name="W_init", dtype=tf.float32)
W = tf.Variable(W_init, name="W")

# Tile the weight matrix 

W_tiled = tf.tile(W, [tf.shape(X)[0], 1, 1, 1, 1], name="W_tiled")

In [42]:
# Expand output of primary capsule (batchsize x 1152 x 8) to (batchsize x 1152 x 10 x 8 x 1) 

primary_capsule_output_expanded = tf.expand_dims(primary_capsule_output, -1, name="primar_capsule_output_expanded")
primary_capsule_output_tile = tf.expand_dims(primary_capsule_output_expanded, 2, name="primary_capsule_output_tile")
primary_capsule_output_tiled = tf.tile(primary_capsule_output_tile, [1, 1, 3, 1, 1], name="primary_capsule_output_tiled")

In [43]:
# Secondary Capsule Prediction Vector uj|i = Wij * uij for all i, j 

secondary_capsule_predicted = tf.matmul(W_tiled, primary_capsule_output_tiled, name="secondary_capsule_prediction")

In [44]:
secondary_capsule_predicted

<KerasTensor: shape=(None, 1152, 3, 16, 1) dtype=float32 (created by layer 'tf.linalg.matmul_3')>